In [1]:
import os
import cv2
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import pandas as pd
from matplotlib.lines import Line2D

In [2]:
# configuration 
IMAGE_DIR    = "../bb_comp"   
COLS, ROWS   = 6, 10
SCALE        = 0.25
BORDER_WIDTH = 40
GUTTER       = BORDER_WIDTH*0.8
GROUP_COLORS = {
    "PTB211" : (1.0, 0.0, 0.0),
    "PTB221" : (0.0, 1.0, 0.0),
    "PTB222" : (0.0, 0.0, 1.0),
    "PTB223" : (1.0, 1.0, 0.0),
}
PATCH_LETTER = "PTB 22.1 - 00"
PATCH_COLOR  = (1.0, 0.853, 0.725, 0.3) 
FONTSIZE     = 7
DPI          = 800

In [3]:
# metadata master key with annotations 
df = pd.read_excel("../data/geomx_annotations_all_batches_only_ptb.xlsx")
counts = pd.read_csv("../data/geomx_ptb_headers.csv",index_col=0)
count_names = [x.split(" | ")[1].replace(" ", "")+ "-"+str(int(x.split(" | ")[2])) for x in counts.columns]

In [4]:
# color mapping for spot frames 
region_color_map = {
    "Alveoli": "#F8766D",
    "Core": "#00BA38",
    "Mantle": "#619CFF"
}


In [5]:
# gather files
files = sorted(
    (f for f in os.listdir(IMAGE_DIR) if f.lower().endswith('.png')),
    key=lambda f: (f.split('_')[0], int(os.path.splitext(f)[0].split('_')[1]))
)

# pixel dimensions of first file 
sample = cv2.imread(os.path.join(IMAGE_DIR, files[0]), cv2.IMREAD_UNCHANGED)
h0, w0 = sample.shape[:2]
tile_w, tile_h = int(w0 * SCALE), int(h0 * SCALE)

In [6]:
# we only want to keep the spots that are alveoli, core, or mantle 
subfiles = []
for fname in files:
    ptb,roi=fname.split("_")
    ptb=ptb[:-1]+"."+ptb[-1]
    roi=int(roi.split(".png")[0])

    if ptb+"-"+str(roi) not in count_names:
        continue

    border_anno = df.loc[(df["Scan name"]==ptb) & (df["ROI"]==roi),"Sub-classification"].values[0]

    # GeoMx spots that were excluded 
    if border_anno in ['Giant-mantle','Fibrosis','Infiltrate']:
        continue

    subfiles += [fname]

In [7]:
fig, axes = plt.subplots(ROWS, COLS,
                         figsize=((tile_w + GUTTER) * COLS / DPI,
                                  (tile_h + GUTTER) * ROWS / DPI),
                         dpi=DPI)
plt.subplots_adjust(wspace=GUTTER/tile_w, hspace=GUTTER/tile_h)

# for each grid element, plot the spot and a frame in its color 
for ax, fname in zip(axes.flat, subfiles):
    ptb,roi=fname.split("_")
    ptb=ptb[:-1]+"."+ptb[-1]
    roi=int(roi.split(".png")[0])

    border_anno = df.loc[(df["Scan name"]==ptb) & (df["ROI"]==roi),"Sub-classification"].values[0]

    # we are considering giant spots as cores 
    if border_anno == 'Giant':
        border_anno = "Core"
    border_color = region_color_map[border_anno]
    
    # load & scale
    img = cv2.imread(os.path.join(IMAGE_DIR, fname), cv2.IMREAD_UNCHANGED)
    img = cv2.resize(img, (tile_w, tile_h), interpolation=cv2.INTER_AREA)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    ax.imshow(
        img,
        extent=(0, 1, 0, 1),
        aspect='auto',
        interpolation='nearest',
        zorder=0
    )
    ax.set_xticks([])
    ax.set_yticks([])
    
    # recolor & thin all four spines to white
    for spine in ax.spines.values():
        spine.set_visible(True)        
        spine.set_edgecolor("white")  
        spine.set_linewidth(0.5)    
        spine.set_zorder(3)   

    
    if roi < 10:
        roi="0"+str(roi)
    else:
        roi=str(roi)
        
    rect = patches.Rectangle((0,0), 1, 1,
                             transform=ax.transAxes,
                             linewidth=BORDER_WIDTH * 1.25 * 72.0 / DPI, 
                             edgecolor=border_color,
                             facecolor="none")
    ax.add_patch(rect)

    # place a title in top left corner of each image 
    ax.text(
        0.0 + 0.075, 1.0 - 0.075,
        ptb[3:] + "-" + roi,
        transform=ax.transAxes,
        fontsize=FONTSIZE,
        fontweight="normal",
        color="white",
        va="top", ha="left"
    )

# turn off axis for last cell to plot legend 
for ax in axes.flat[len(subfiles):]:
    ax.axis("off")


legend_labels = ["Alveoli","Core", "Mantle"]
legend_ax = axes.flat[len(subfiles)]
legend_handles = [
    Line2D([0], [0], marker='s', color='none', label=legend_labels[0],
           markerfacecolor=region_color_map[legend_labels[0]], markersize=10, markeredgecolor='white'),
    Line2D([0], [0], marker='s', color='none', label=legend_labels[1],
           markerfacecolor=region_color_map[legend_labels[1]], markersize=10, markeredgecolor='white'),
    Line2D([0], [0], marker='s', color='none', label=legend_labels[2],
           markerfacecolor=region_color_map[legend_labels[2]], markersize=10, markeredgecolor='white')
]

legend_ax.legend(
    handles=legend_handles,
    loc='center',
    fontsize=FONTSIZE*1.8,
    markerscale=1.5,
    frameon=False,
    labelspacing=1
)

plt.savefig("../plots/composite_ACM.png", dpi=DPI, bbox_inches="tight")
plt.close(fig)
plt.clf()

<Figure size 640x480 with 0 Axes>